# Preparação de Dados (N1)

---

## 🗂️ Projeto Telco Customer Churn Analysis  

**Dataset:** Telcom Customer Churn (7043 registros, 21 variáveis)  
**Objetivo Geral:** Analisar o comportamento de cancelamento de clientes em uma empresa de telecomunicações, aplicando técnicas de Machine Learning para prever o churn e identificar fatores que influenciam a retenção.

---

## 📋 Descrição Geral do Tratamento

Nesta etapa serão realizados os principais **processos de limpeza, transformação e padronização dos dados**.  
O foco é garantir que o dataset esteja **consistente, estruturado e pronto para análises posteriores**.

**Tarefas previstas:**  
- Leitura e diagnóstico inicial do dataset.  
- Tratamento de valores nulos, duplicados e outliers.  
- Conversão de tipos e padronização de nomes de colunas.  
- Codificação de variáveis categóricas e normalização de numéricas (quando necessário).  
- Geração de artefatos intermediários e relatório de qualidade dos dados.

## 🔧 Configuração do Projeto (Bootstrap)

Esta etapa realiza o **bootstrap do ambiente de execução**, preparando o notebook para funcionar em qualquer máquina — garantindo **reprodutibilidade**, **organização** e **autonomia**.  
Todas as dependências, caminhos e configurações são ajustadas automaticamente, sem necessidade de modificações manuais.

---

### ⚙️ Etapas executadas

#### 🧭 1. Localização da raiz do projeto  
- Busca o arquivo `config/defaults.json` subindo diretórios até encontrá-lo.  
- Define `PROJECT_ROOT` como referência principal para todo o projeto.

#### 🧩 2. Validação do módulo `utils/`  
- Verifica a existência da pasta `utils/` e do arquivo `__init__.py`.  
- Cria o arquivo caso ausente, garantindo que o pacote seja importável.

#### 🧱 3. Registro no `sys.path`  
- Injeta o diretório raiz no `sys.path`.  
- Permite importar módulos internos (`utils.utils_data`) em qualquer ambiente.

#### ♻️ 4. Importação das utilidades  
- Importa e recarrega `utils.utils_data` via `importlib.reload()`.  
- Garante que a versão mais recente do módulo seja utilizada no notebook.

#### 🧾 5. Inicialização de logs  
- Configura o sistema de logging unificado (`reports/data_preparation.log`).  
- Todos os eventos são registrados no console e em arquivo.

#### ⚙️ 6. Carregamento de configurações  
- Lê os parâmetros de `defaults.json` e aplica substituições de `local.json` (se houver).  
- Resolve automaticamente os caminhos padrão do N1.

#### 🔄 7. Reprodutibilidade global e ajustes finais de ambiente  
- Define a semente aleatória (`RANDOM_SEED`) para NumPy, Random e variáveis de ambiente.  
- Assegura resultados reprodutíveis entre diferentes execuções. 
- Exibe confirmação no log sobre configuração e paths resolvidos.

---

### ✅ Resultado esperado

- O ambiente é inicializado corretamente em qualquer máquina.  
- O módulo `utils_data.py` é importado e recarregado.  
- As pastas e arquivos padrão são criados (se necessário).  
- O log central passa a registrar todas as ações subsequentes.

---

> 💡 **Resumo:** Esta célula garante que o notebook seja totalmente **portável e reprodutível**, sem necessidade de alterar caminhos ou dependências manualmente.


In [1]:
# -*- coding: utf-8 -*-
# Bootstrap do projeto — encontra a raiz, injeta no sys.path e garante utils/__init__.py

from IPython.display import display
from pathlib import Path
import sys
import logging
from datetime import datetime
import importlib
import pandas as pd
import json
import os
import random
import numpy as np

# ---------------------------------------------------------
# Helpers mínimos
# ---------------------------------------------------------
def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

# 1) Descobrir a raiz a partir do config/defaults.json
_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError("config/defaults.json não encontrado. Confirme a estrutura do projeto.")
PROJECT_ROOT = _cfg.parent.parent.resolve()
print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")

# 2) Garantir pasta utils/ e __init__.py
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"
if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR} (crie 'utils' na raiz).")
if not INIT_FILE.exists():
    INIT_FILE.write_text("", encoding="utf-8")
    print(f"[INFO] Criado: {INIT_FILE}")

# 3) Injetar a raiz no sys.path
root_str = str(PROJECT_ROOT)
if root_str not in sys.path:
    sys.path.insert(0, root_str)
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

# 4) Importar utils.utils_data
import utils.utils_data as ud
importlib.reload(ud)  # garante versão mais recente ao iterar no notebook

# 5) Configurar logging base do notebook
LOG_FILE = (PROJECT_ROOT / "reports" / "data_preparation.log")
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler(LOG_FILE, encoding="utf-8")],
    force=True,
)
logger = logging.getLogger(__name__)
logger.info("Bootstrap iniciado…")

# 6) Carregar configurações
config = ud.load_config(PROJECT_ROOT / "config" / "defaults.json",
                        PROJECT_ROOT / "config" / "local.json")

paths = ud.resolve_n1_paths(PROJECT_ROOT)

# 7) Reprodutibilidade (seed) e display
RANDOM_SEED = config.get("random_seed", 42)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 120)

logger.info("Bootstrap concluído. Config carregada e paths resolvidos.")

[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils
2025-11-13 21:27:05,375 | INFO | Bootstrap iniciado…
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:05,377 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils
2025-11-13 21:27:05,379 | INFO | sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils
2025-11-13 21:27:05,383 | INFO | Bootstrap concluído. Config carregada e paths resolvidos.


## 🔧 Configuração de Fontes

Nesta etapa são definidos os **arquivos de origem** que servirão de base para o projeto.  
Aqui é onde especifica **quais datasets serão utilizados**, **em qual formato** estão (CSV ou Parquet) e, caso existam múltiplas fontes, **como elas se relacionam** entre si.

---

### 🗂️ 1. Exploração das fontes disponíveis

Antes de configurar o dicionário `SOURCES`, é possível listar os arquivos presentes no diretório `data/raw/`:

```python
from utils.utils_data import list_raw_sources_safe

RAW_DIR = paths["raw_dir"] if isinstance(paths, dict) else paths.raw_dir
df_sources = list_raw_sources_safe(RAW_DIR, pattern="*.csv", show_rel=True)
```

Esse comando exibe uma tabela com os arquivos encontrados e gera uma **sugestão automática de caminho** para uso no `SOURCES`.  
Basta mover o arquivo desejado para a pasta `data/raw/` e executar o bloco — o nome aparecerá pronto para ser copiado e colado.

---

### ⚙️ 2. Definição das fontes de dados (`SOURCES`)

- Cada entrada do dicionário `SOURCES` representa uma **tabela nomeada**.  
- A chave (ex.: `"main"`, `"dim_customers"`) identifica a tabela dentro do projeto.  
- Cada item deve conter:
  - `path`: caminho do arquivo dentro de `data/raw/`;  
  - `format`: formato opcional (`"csv"` ou `"parquet"`, detectado automaticamente se omitido);  
  - `read_opts`: parâmetros personalizados de leitura (`encoding`, `sep`, `low_memory`, etc.).

Exemplo de estrutura:
```python
SOURCES = {
    "main": {
        "path": RAW_DIR / "dataset.csv",
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo adicional em CSV:
     "dim_customers": {
         "path": RAW_DIR / "exemplo.csv",
         "format": "csv"
     },
    # Exemplo em Parquet:
     "dim_customers": {
         "path": RAW_DIR / "customers.parquet",
         "format": "parquet"
     }
}
```

---

### 🔗 3. Configuração opcional de junções (`MERGE_STEPS`)

O DataFrame principal é aquele definido em `MAIN_SOURCE`.  
Se existirem outras fontes, é possível configurá-las em `MERGE_STEPS` para realizar junções automáticas:

```python
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]
```

Cada item representa uma junção e define:
- o nome da tabela secundária (`right_name`),  
- o tipo de junção (`how`),  
- e as chaves de correspondência (`left_on`, `right_on`).

---

### ✅ 4. Validação e feedback automático

Após definir as fontes, o bloco de código executa uma checagem simples para:

- Confirmar a existência do arquivo principal;  
- Detectar automaticamente o formato (`CSV` ou `Parquet`);  
- Exibir mensagens de status:

```
✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: dataset.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).
```

---

> 💡 **Resumo**
>
>Essa abordagem torna a **configuração de fontes interativa, leve e flexível**:
>- Basta adicionar novos arquivos em `data/raw/` e executar `suggest_source_path()` para gerar os caminhos automaticamente.  
>- Nenhum outro bloco do notebook precisa ser alterado — apenas o dicionário `SOURCES` e, se necessário, as junções em `MERGE_STEPS`.

---

📂 **Abaixo, bloco auxiliar:**  
Para visualizar as fontes disponíveis em `data/raw/`, execute:

In [2]:
from utils.utils_data import list_raw_sources_safe
RAW_DIR = paths["raw_dir"] if isinstance(paths, dict) else paths.raw_dir
display(list_raw_sources_safe(RAW_DIR, pattern="*.csv", show_rel=True))

,file,size,size_bytes,modified,relpath
0,dataset.csv,955 KB,977501,2019-09-27T19:30:08,data\raw\dataset.csv
1,WA_Fn-UseC_-Telco-Customer-Churn.csv,955 KB,977501,2019-09-27T19:30:08,data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv


In [3]:
#from utils.utils_data import show_source_overview_neat, show_df_summary_neat, run_quality_and_typing, render_quality_and_typing

# Obs: só alterar este bloco quando precisar mudar os arquivos ou as chaves de junção.

SOURCES = {
    # nome_da_tabela: {path, (opcional) format: 'csv'|'parquet', (opcional) read_opts}
    "main": {
        "path": RAW_DIR / "WA_Fn-UseC_-Telco-Customer-Churn.csv",
        # "format": "csv",  # se omitir, detecta pelo sufixo
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #   "path": RAW_DIR / "exemplo.csv",
    #   "format": "csv",  # se omitir, detecta pelo sufixo
    #   "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    # }
    
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}

# Plano de merges (opcional):
# Cada item é um passo: (right_name, how, left_on, right_on)
# O DataFrame base será o SOURCES[MAIN_SOURCE]
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]

# Validação
main_path = SOURCES[MAIN_SOURCE]["path"]
main_format = SOURCES[MAIN_SOURCE].get("format") or main_path.suffix.lstrip(".").lower()
print(f"✅ Fontes configuradas com sucesso.")

print(
    f"→ Fonte principal: '{MAIN_SOURCE}'  "
    f"| Arquivo: {main_path.name}  "
    f"| Formato detectado: {main_format.upper()}"
)
if MERGE_STEPS:
    print(f"→ {len(MERGE_STEPS)} etapa(s) de merge configuradas.")
else:
    print("→ Nenhum merge configurado (usando apenas a fonte principal).")


✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: WA_Fn-UseC_-Telco-Customer-Churn.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).


# 📥 Ingestão & Visão Rápida

Nesta etapa, os **datasets definidos em `SOURCES`** são carregados, inspecionados e validados antes de iniciar o tratamento dos dados.  
O objetivo é garantir que as fontes estejam corretamente lidas e que o DataFrame principal (`df`) esteja pronto para seguir no pipeline.

---

## ⚙️ 1) Carregamento das fontes

O notebook percorre cada item de `SOURCES` para:
- Ler o arquivo (`path`) conforme o formato (`csv` ou `parquet`);
- Aplicar as opções de leitura (`read_opts`), quando houver;
- Armazenar o DataFrame em `tables[name]`, onde `name` é a chave da fonte.

A função usada é `ud.load_table_simple()` — ela **detecta o formato pelo sufixo** (se não informado) e aplica `read_opts` com segurança.

Após cada leitura, é mostrada uma visão compacta da fonte via `show_source_overview(name, path, df_src)`, com:
- Linhas, colunas e memória aproximada em MB;
- Resumo de tipos (`dtype`);
- Top 20 colunas com maiores índices de valores ausentes.

> **Nota:** Esta visão é **por fonte**. Se houver múltiplas fontes, você verá um bloco para cada uma.

---

## 🔗 2) Definição do DataFrame base e junções (opcional)

O dataset principal é `tables[MAIN_SOURCE]`.  
Se houver fontes adicionais e `MERGE_STEPS` definido, o notebook aplica as junções com `ud.merge_chain()` (ou lógica equivalente), registrando logs úteis para depuração.  
Sem junções, o fluxo segue apenas com a fonte principal.

---

## 📊 3) Visão consolidada do DataFrame que segue no pipeline

Depois do carregamento (e merges, se houver), exibimos **uma segunda visão** via `show_df_summary(df, label=...)`.  
Ela é **parecida** com a visão por fonte, mas agora representa **o DataFrame resultante** que seguirá para as próximas etapas (limpeza, tipagem, etc.).

- Estrutura geral (linhas/colunas/memória);
- Resumo de tipos;
- Top 20 de faltantes no **df final** (após merges, se configurados).

---

> 💡  Resumo
> - **Visão por fonte** (um card por item de `SOURCES`) ajuda a detectar problemas na **leitura individual** (tipos inesperados, ausências, tamanho/atualização do arquivo).  
> - **Visão consolidada** confirma o que **segue no pipeline**; se houver merges, é aqui que aparecem os efeitos combinados.  
> - Logs e relatórios são gravados para apoiar **rastreabilidade** e **reprodutibilidade**.


In [4]:
# Ingestão & visão rápida — organizado (autônomo)

from utils.utils_data import (
    load_table_simple, merge_chain,
    show_source_overview, show_df_summary
)

# 1) Carregar todas as fontes em 'tables'
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")               # None => infere pelo sufixo
    read_opts = cfg.get("read_opts", {})  # ex.: sep/encoding/low_memory para CSV
    df_src = load_table_simple(path, fmt, **read_opts)
    tables[name] = df_src

    # visão organizada por fonte (cards)
    show_source_overview(name, path, df_src)

# 2) Definir df base e aplicar merges (se configurado)
if MAIN_SOURCE not in tables:
    raise KeyError(f"MAIN_SOURCE '{MAIN_SOURCE}' não encontrado. Fontes disponíveis: {list(tables.keys())}")

df = tables[MAIN_SOURCE]

# compat: aceita MERGE_STEPS como lista de tuplas OU de dicts
def _normalize_steps(steps):
    norm = []
    for st in (steps or []):
        if isinstance(st, (list, tuple)) and len(st) >= 4:
            right_name, how, left_on, right_on = st[:4]
            norm.append({"from": right_name, "how": how, "left_on": left_on, "right_on": right_on})
        elif isinstance(st, dict):
            norm.append(st)
    return norm

if MERGE_STEPS:
    steps = _normalize_steps(MERGE_STEPS)
    if steps:
        df = merge_chain(df, tables, steps)
        print("\n=== Visão geral (df após merges) ===")
    else:
        print("\n[WARN] MERGE_STEPS definido mas vazio/inválido; seguindo sem merge.")
else:
    print(f"\n[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# 3) Resumo organizado do df que segue no pipeline (cards)
show_df_summary(df, label=f"MAIN_SOURCE = {MAIN_SOURCE}")


,Métrica,Valor
0,Linhas,7043
1,Colunas,21
2,Memória (MB),6.821


,dtype,cols
0,object,18
1,int64,2
2,float64,1


,Coluna,dtype,Faltantes,%Faltantes
0,customerID,object,0,0
1,gender,object,0,0
2,SeniorCitizen,int64,0,0
3,Partner,object,0,0
4,Dependents,object,0,0
5,tenure,int64,0,0
6,PhoneService,object,0,0
7,MultipleLines,object,0,0
8,InternetService,object,0,0
9,OnlineSecurity,object,0,0



[INFO] Usando df base: 'main' (sem merges).


,Métrica,Valor
0,Linhas,7043
1,Colunas,21
2,Memória (MB),6.821


,dtype,cols
0,object,18
1,int64,2
2,float64,1


,Coluna,dtype,Faltantes,%Faltantes
0,customerID,object,0,0
1,gender,object,0,0
2,SeniorCitizen,int64,0,0
3,Partner,object,0,0
4,Dependents,object,0,0
5,tenure,int64,0,0
6,PhoneService,object,0,0
7,MultipleLines,object,0,0
8,InternetService,object,0,0
9,OnlineSecurity,object,0,0


# 🧭 Catálogo de DataFrames e Definição do df “Ativo”

Nesta etapa é criado um **catálogo centralizado de DataFrames** para organizar e gerenciar todas as tabelas carregadas e derivadas ao longo do projeto.  
O objetivo é garantir **reprodutibilidade, rastreabilidade e controle de versões** das diferentes fases do pipeline de dados.

---

## ⚙️ 1. Inicialização do Catálogo (`TableStore`)

A classe `TableStore` — implementada em `utils/utils_data.py` — funciona como um **repositório de DataFrames nomeados**.  
Ela é inicializada com o dicionário `tables` (gerado na etapa de *Ingestão & Visão Rápida*) e define a tabela principal (`MAIN_SOURCE`) como **tabela ativa**.

Durante a execução, é exibido um feedback informativo, como no exemplo abaixo:

```
✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7043 linhas × 21 colunas
```

E, logo em seguida, uma prévia do inventário atual:

| name | rows | cols | memory_mb | current |
|------|------|-------|-----------|----------|
| main | 7043 | 21 | 6.821 | ✅ |

---

## 📊 2. Definição do DataFrame Ativo (`df`)

A variável `df` é atribuída à tabela atual com o comando `T.get()`.  
A partir desse ponto, `df` representa o **DataFrame ativo** — ou seja, o dataset que segue para as próximas etapas do pipeline  
(limpeza, tipagem, tratamento de faltantes, engenharia de atributos, etc.).

---

## 🧩 3. Gerenciamento de Tabelas no Catálogo

Novos DataFrames podem ser adicionados ao catálogo a qualquer momento, com nomes descritivos e controle de versão.  
O `TableStore` também permite alternar entre tabelas existentes, consultar o catálogo e acessar DataFrames diretamente pelo nome.

**Exemplos práticos:**
```python
T.add("churn_raw", df, set_current=True)           # adiciona e define como atual
df = T.use("churn_raw")                            # alterna o df ativo
T.add("features_v1", engenharia_de_atributos(df))  # salva uma derivação
df_features = T["features_v1"]                     # acesso direto por nome
display(T.list())                                  # inventário completo
```

---

## 🧠 4. Benefícios Práticos

- Mantém o notebook **organizado**, sem sobrescrever DataFrames intermediários.  
- Permite **comparar diferentes versões** de tratamento ou feature engineering.  
- Facilita o **debug** e o **reprocessamento seletivo** de etapas específicas.  
- Garante **visibilidade** sobre o estado atual do pipeline, com feedback visual e logs consistentes.

---

> 💡 **Resumo:**  
> O `TableStore` atua como uma **memória estruturada** de DataFrames —  
> permitindo **adicionar**, **alternar**, **versionar** e **consultar** tabelas de forma controlada.  
>  
> A variável `df` representa sempre a **tabela ativa atual**, garantindo consistência e rastreabilidade em todas as etapas subsequentes do pipeline.


In [5]:
from utils.utils_data import TableStore

# 1- Inicializa catálogo com as fontes lidas ('tables') e define a base como atual
T = TableStore(initial=tables, current=MAIN_SOURCE)

# 2 - Conveniência: df = tabela atual (boa prática para seguir no pipeline)
df = T.get()

# 3 - Feedback visual de sucesso
print(f"✅ Catálogo de DataFrames inicializado com sucesso.")
print(f"→ Total de tabelas carregadas: {len(tables)}")
print(f"→ Tabela ativa: '{T.current}'  | Shape: {df.shape[0]} linhas × {df.shape[1]} colunas")

# 4️4 - Exibe uma prévia do catálogo
display(T.list().head())

# --- Guia rápido  ---
# T.add("churn_raw", df, set_current=True)          # cadastra e ativa
# df = T.use("churn_raw")                           # muda o atual e retorna df
# T.add("features_v1", engenharia_de_atributos(df)) # salva uma derivação
# df_features = T["features_v1"]                    # acesso direto por nome
# display(T.list())                                 # inventário das tabelas

✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7043 linhas × 21 colunas


,name,rows,cols,memory_mb,current
0,main,7043,21,6.821,True


In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 🧪 Qualidade & Tipagem

Nesta etapa o notebook realiza a **padronização estrutural e tipagem automática dos dados**, garantindo que o DataFrame principal (`df`) siga para as próximas fases **coerente, limpo e otimizado em memória**.  
É uma das fases mais críticas do pipeline, pois evita que inconsistências de tipo e formato prejudiquem as análises posteriores ou a modelagem.

### 📋 O que é feito aqui

1. **Remoção de espaços em branco (`strip_whitespace`)**
   - Elimina espaços extras no início e no fim de valores textuais.  
   - Evita divergências em comparações e agrupamentos (ex.: `"Yes "` ≠ `"Yes"`).  
   - Essa operação é **idempotente**: se os dados já estiverem limpos, não altera nada.

2. **Conversão inteligente de valores numéricos (`infer_numeric_like`)**
   - Detecta colunas com valores *aparentemente numéricos*, mas armazenados como texto (ex.: `"R$ 120,00"` ou `"1.234,56"`).  
   - Aplica heurísticas para remover símbolos, normalizar separadores e converter apenas quando a proporção de conversão for suficiente (`min_ratio = 0.9`).  
   - Evita conversões indevidas (ex.: IDs) por meio de uma lista de exclusão (`blacklist`).  
   - Gera o relatório **`cast_report`**, registrando:
     - Coluna analisada  
     - Ação tomada (convertida, ignorada ou parcial)  
     - Taxa de sucesso e total de valores convertidos.

3. **Otimização de tipos numéricos (`reduce_memory_usage`)**
   - Converte automaticamente `int64` → `int32` e `float64` → `float32`, reduzindo o consumo de memória sem alterar os valores.  
   - O log mostra a diferença antes e depois (ex.: `Memory reduced: 7.76MB → 6.21MB`).

4. **Remoção de duplicatas (`deduplicate_rows`)**
   - Verifica e elimina registros repetidos conforme a configuração (`subset`, `keep`).  
   - Caso encontre duplicatas, gera um relatório detalhado e salva o log em `reports/duplicates.csv`.  
   - Exibe uma amostra das duplicatas detectadas, se houver.

5. **Relatórios e feedback automático**
   - Ao final da execução, exibe um resumo completo:  
     - Linhas e colunas antes/depois  
     - Variação de memória (`Δ MB`)  
     - Relatórios gerados (`cast_report`, `duplicates_report`, etc.)  
   - Todos os logs são gravados em `reports/data_preparation.log`.

---

> 💡 **Resumo:**  
> Esta célula garante que o dataset esteja **consistente, tipado e otimizado**, pronto para análises e modelagem.  
> Corrige formatações, converte números armazenados como texto, remove duplicatas e reduz o uso de memória — tudo com registros automáticos e relatórios salvos para auditoria.


In [7]:
# 1) Qualidade & Tipagem
_result = ud.run_quality_and_typing(df, config)
df = _result["df"]
ud.render_quality_and_typing(_result)

# 2) Corrige avg_charge_per_month quando tenure == 0
df = ud.fix_avg_charge_zero_tenure(df)

# 3) Preenche TotalCharges e outras numéricas com flag
df, _nullmeta = ud.null_fill_from_config(df, config)
ud.render_null_fill_report(_nullmeta)

# 4) Recalcula charge_gap e charge_gap_log1p com base nas colunas corrigidas
df = ud.recompute_charge_gap_features(df)


,Métrica,Antes,Depois,Δ
0,Linhas,7043,7043,0
1,Colunas,21,21,0
2,Memória,6.82 MB,6.51 MB,-0.31 MB


,column,converted_non_null,introduced_nans,dtype_after
16,TotalCharges,7032,11,float64


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:07,314 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:07,320 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:07,351 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [manifest] step='save_report_df' registrado.
2025-11-13 21:27:07,369 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\nulls\fill_summary.csv (1 linhas)
2025-11-13 21:27:07,373 | INFO | [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-

• Colunas preenchidas: 1
  - TotalCharges
• Flags criadas      : 1
• Relatório (comparativo antes/depois) salvo em: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\nulls\fill_summary.csv


,column,missing_count,missing_pct
0,TotalCharges,11,0.16


# 🧼 Padronização Categórica

Nesta etapa ocorre a **padronização global de rótulos textuais** em colunas categóricas, corrigindo inconsistências, diferenças de capitalização e valores redundantes (ex.: `"no internet service"` → `"no"`).  
O objetivo é **garantir uniformidade sem alterar o significado dos dados**, preparando o dataset para codificação e modelagem.

---

## 🧭 Fase Zero — Descobrir padrões categóricos (candidatos)
Antes de padronizar, o notebook **sugere colunas candidatas** via heurísticas e exibe *cards* com:
- **Top candidatos** (prioridade por suspeita e baixa cardinalidade)
- **Provavelmente binárias (Yes/No)**
- **Frases de serviço** (ex.: `"no internet service"`, `"no phone service"`)

Use isso para **ajustar seu `CAT_NORM_CFG`** de forma mais fluida.

**Bloco de uso**:
```python
import utils.utils_data as ud

# Sugere e renderiza cards (salva CSVs em reports/categorical_candidates/ se paths.reports_dir existir)
ud.render_categorical_candidates(
    df,
    max_unique_ratio=0.5,
    max_unique_count=50,
    include_numeric_small=True,   # ou False para evitar falsos-positivos numéricos (ex.: tenure==1)
    # base_dir=paths.reports_dir, # opcional: persistir CSVs
    top_n=30,
    head_bin=20,
    head_service=20,
)
```

> **Dica:** Se apareceram colunas numéricas como “binárias” por conter valor `1`, reexecute com `include_numeric_small=False` para limpar o ruído.

---

## 📋 O que é feito na padronização

1. **Seleção automática das colunas categóricas**  
   - Processa colunas com tipo `object`, `string` ou `category`, exceto as listadas em `exclude` (ex.: `"customerID"`).

2. **Limpeza e normalização de forma**  
   - Remove espaços extras (`trim`) e consolida espaços duplos (`collapse_ws`).  
   - Corrige capitalização conforme `case` (`"lower"`, `"upper"`, `"title"`, `"none"`).  
   - Opcionalmente remove acentos (`strip_accents`).

3. **Substituições e valores nulos**  
   - Aplica **mapa global** (`global_map`) para corrigir padrões (ex.: `"no internet service"` → `"no"`).  
   - Suporta **mapa por coluna** (`per_column_map`) para tratamentos finos.  
   - Converte valores definidos em `null_values` (ex.: `"n/a"`, `"na"`, `"-"`) em `NaN`.

4. **Conversão opcional para `category`**  
   - Se `cast_to_category=True`, converte colunas textuais em `category`, otimizando memória.

5. **Relatório e registro automático**  
   - Gera `reports/cat_normalization.csv` com: coluna, alterações (`changed`), etc.  
   - Exibe cards com **impacto** (linhas/colunas/memória) e **top mudanças**.  
   - Integração com `manifest.json` quando disponível.

---

## ⚙️ Configuração recomendada (`CAT_NORM_CFG`)

> **Importante sobre *case***: O mapeamento em `global_map` é por texto exato.  
> Para evitar desencontro (ex.: `"No internet service"` virar `"No Internet Service"` antes do mapeamento), recomenda‑se usar **`case: "lower"`** e **chaves em minúsculas** no `global_map`.

```python
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],   # nunca normalizar IDs
    "case": "lower",             # recomendação para mapeamento determinístico
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "no internet service": "no",
        "no phone service": "no",
        "n/a": "no",
        "none": "no"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False,
    # "per_column_map": {"PaperlessBilling": {"sim": "yes", "nao": "no"}}
}
```

**Exemplo de mapa específico por coluna:**
```python
"per_column_map": {
    "InternetService": {"dsl": "dsl", "fiber optic": "fiber optic", "no": "no"}
}
```

---

## 🔎 Validação rápida pós-normalização
Confirme que as colunas “tri-estado” foram reduzidas a **rótulos coerentes** (ex.: `yes/no`):

```python
cols_tri = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",
            "StreamingTV","StreamingMovies","MultipleLines","InternetService"]
for c in cols_tri:
    print(f"\n{c} — value_counts():")
    print(df[c].value_counts(dropna=False).head(10))
```

Se ainda aparecer `"no internet service"`/`"no phone service"`, verifique:
- Se `case` está como `"lower"` e as chaves do `global_map` também;
- Se `exclude` não está retirando a coluna do fluxo;
- Se há um `per_column_map` sobrescrevendo o `global_map`.

---

> 💡  Resumo
>A padronização categórica é **inteligente e auditável**: aplica limpeza de forma, substituições globais/específicas e pode reduzir memória convertendo para `category`.  
> Com a **Fase Zero de Descoberta**, você ajusta o `CAT_NORM_CFG` com confiança antes da normalização.  
> Relatórios, logs e manifest garantem **clareza, rastreabilidade e consistência** ao longo do pipeline.


In [8]:
# ——— Candidatos à Padronização Categórica (cards + CSVs) ———
ud.render_categorical_candidates(
    df,
    # Se quiser mudar os limites:
    max_unique_ratio=0.5,
    max_unique_count=50,
    include_numeric_small=True,
    # Para salvar CSVs, opcionalmente passe uma pasta base (senão tenta usar paths.reports_dir)
    # base_dir=paths.reports_dir,
    top_n=30,
    head_bin=20,
    head_service=20,
)

,Métrica,Valor
0,Total de colunas,24
1,Candidatas (suspected=True),20
2,Binárias (sugeridas Yes/No),16
3,Com frases de serviço,7


,column,dtype,n_unique,pct_unique,sample_values,reasons
0,gender,object,2,0.0003,"[Male, Female]",texto/categoria
1,SeniorCitizen,int64,2,0.0003,"[0, 1]","numérico baixa cardinalidade, binário (yes/no)"
2,Partner,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
3,Dependents,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
4,PhoneService,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
5,PaperlessBilling,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
6,Churn,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
7,TotalCharges_was_missing,int64,2,0.0003,"[0, 1]","numérico baixa cardinalidade, binário (yes/no)"
8,MultipleLines,object,3,0.0004,"[No, Yes, No phone service]","texto/categoria, binário (yes/no), frases de s..."
9,InternetService,object,3,0.0004,"[Fiber optic, DSL, No]","texto/categoria, binário (yes/no)"


,column,dtype,n_unique,pct_unique,sample_values,reasons
1,SeniorCitizen,int64,2,0.0003,"[0, 1]","numérico baixa cardinalidade, binário (yes/no)"
2,Partner,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
3,Dependents,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
4,PhoneService,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
5,PaperlessBilling,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
6,Churn,object,2,0.0003,"[No, Yes]","texto/categoria, binário (yes/no)"
7,TotalCharges_was_missing,int64,2,0.0003,"[0, 1]","numérico baixa cardinalidade, binário (yes/no)"
8,MultipleLines,object,3,0.0004,"[No, Yes, No phone service]","texto/categoria, binário (yes/no), frases de s..."
9,InternetService,object,3,0.0004,"[Fiber optic, DSL, No]","texto/categoria, binário (yes/no)"
10,OnlineSecurity,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."


,column,dtype,n_unique,pct_unique,sample_values,reasons
8,MultipleLines,object,3,0.0004,"[No, Yes, No phone service]","texto/categoria, binário (yes/no), frases de s..."
10,OnlineSecurity,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."
11,OnlineBackup,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."
12,DeviceProtection,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."
13,TechSupport,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."
14,StreamingTV,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."
15,StreamingMovies,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, binário (yes/no), frases de s..."


In [9]:
# ——— Padronização Categórica ———

# Config genérica do template (ajuste à vontade)
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],          # nunca normalizar IDs
    "case": "lower",                    # "lower" | "upper" | "title"
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "no internet service": "no",    
        "no phone service": "no",
        "n/a": "no",
        "none": "no"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False,
    # "per_column_map": {"PaperlessBilling": {"sim": "Yes", "nao": "No"}}
}

# Caminho do relatório (se tiver 'paths', mantenha a convenção)
cat_report_path = paths.reports_dir / "cat_normalization.csv"

# Executa e renderiza
_cat = ud.run_categorical_normalization(df, cfg=CAT_NORM_CFG, report_path=cat_report_path)
df = _cat["df"]
ud.render_categorical_normalization(_cat)


,Métrica,Antes,Depois,Δ
0,Linhas,7043,7043,0
1,Colunas,24,24,0
2,Memória,6.67 MB,6.51 MB,-0.16 MB


,column,changed
0,gender,7043
1,Partner,7043
2,Dependents,7043
3,PhoneService,7043
4,MultipleLines,7043
5,InternetService,7043
6,OnlineSecurity,7043
7,OnlineBackup,7043
8,DeviceProtection,7043
9,TechSupport,7043


In [10]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TotalCharges_was_missing,charge_gap,charge_gap_log1p
0,7590-VHVEG,female,0,yes,no,1,no,no,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no,0,0.00,0.000000
1,5575-GNVDE,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one year,no,mailed check,56.95,1889.50,no,0,-46.80,-3.867026
2,3668-QPYBK,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes,0,0.45,0.371564
3,7795-CFOCW,male,0,no,no,45,no,no,dsl,yes,no,yes,yes,no,no,one year,no,bank transfer (automatic),42.30,1840.75,no,0,-62.75,-4.154969
4,9237-HQITU,female,0,no,no,2,yes,no,fiber optic,no,no,no,no,no,no,month-to-month,yes,electronic check,70.70,151.65,yes,0,10.25,2.420368


## 🩹 Tratamento de Valores Faltantes

Nesta etapa são tratadas as **ausências de dados** (valores nulos ou `NaN`) para garantir que o DataFrame siga consistente para as próximas fases do pipeline.  
O objetivo é **preencher ou sinalizar valores faltantes** de forma controlada, preservando a integridade estatística das colunas e registrando onde ocorreram substituições.

---

### 📋 O que é feito aqui

1. **Geração de relatório inicial**
   - Exibe um diagnóstico das colunas com valores ausentes, mostrando a proporção (`missing_pct`) e o total (`missing_count`) de registros nulos.
   - Esses relatórios são salvos automaticamente em `reports/missing/before.csv`.

2. **Aplicação da estratégia de imputação**
   - O comportamento é controlado pela configuração `config["missing"]` e pelo parâmetro `prefer="auto"`.
   - Quando `prefer="auto"`, o sistema escolhe a estratégia disponível (por padrão, `"simple"`).  
   - A estratégia **simples** executa:
     - Substituição de valores nulos **numéricos** pela **mediana** da coluna.  
     - Substituição de valores nulos **categóricos** pela **moda** (valor mais frequente).  
   - Durante o processo, são criadas **colunas de flag** no formato `<coluna>_was_missing`, indicando quais células estavam ausentes originalmente.

3. **Relatório final e auditoria**
   - Após o preenchimento, é exibido novamente o relatório de faltantes (`reports/missing/after.csv`) e cards de verificação visual.
   - Caso ainda existam colunas críticas, o notebook pode ser ajustado para aplicar métodos mais sofisticados.

---

### 🧾 Exemplo de flag gerada

| customerID | TotalCharges | TotalCharges_was_missing |
|-------------|--------------|--------------------------|
| 7590-VHVEG  | 29.85        | False                   |
| 9237-HQITU  | 1840.75      | True                    |

Linhas com `True` indicam que o valor original estava ausente antes da imputação — servindo como **marca de auditoria** e controle de qualidade.

---

### 📊 Visualização e Relatórios

Após a execução, o notebook renderiza automaticamente um **painel de diagnóstico**, com:

- **Resumo da etapa:** número de linhas, colunas, estratégia aplicada e flags criadas.  
- **Top colunas com faltantes (antes)** — ajuda a identificar onde a imputação foi mais significativa.  
- **Verificação pós-imputação** — confirma se ainda existem valores ausentes.  
- **Amostra das flags `_was_missing`** criadas.  

Esses relatórios garantem rastreabilidade e transparência no pipeline.

---

> 💡 **Resumo:**  
> Esta célula identifica e corrige valores faltantes, aplicando regras de imputação simples e registrando as células originalmente ausentes por meio de colunas `*_was_missing`.  
> Os relatórios antes/depois e os cards visuais asseguram **clareza, rastreabilidade e controle de qualidade** em toda a etapa.


In [11]:
# === N1 · Tratamento de Valores Faltantes ===============

df = ud.coerce_df(df)

res = ud.handle_missing_step(
    df,
    config=config,
    save_reports=True,
    prefer="auto"
)
df = res["df"]

# Renderização estilizada
ud.render_missing_step(res, df)

[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:08,650 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:08,656 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:08,686 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [manifest] step='save_report_df' registrado.
2025-11-13 21:27:08,708 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\missing\before.csv (24 linhas)
2025-11-13 21:27:08,712 | INFO | [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-cus

,Métrica,Valor
0,Estratégia,simple
1,Linhas no df,7043
2,Colunas no df,46
3,Total de valores faltantes (antes),0
4,Colunas com faltantes (antes),0
5,Flags criadas (_was_missing),23


,column,missing_count,missing_pct
45,Churn_was_missing,0,0.0
19,Contract_was_missing,0,0.0
9,Dependents_was_missing,0,0.0
15,DeviceProtection_was_missing,0,0.0
11,InternetService_was_missing,0,0.0
3,MonthlyCharges_was_missing,0,0.0
1,MultipleLines_was_missing,0,0.0
14,OnlineBackup_was_missing,0,0.0
13,OnlineSecurity_was_missing,0,0.0
20,PaperlessBilling_was_missing,0,0.0


## 🚩 Detecção de Outliers

Esta etapa identifica **valores atípicos** nas colunas numéricas contínuas, adicionando colunas de flag (`*_is_outlier`) que indicam quais registros se desviam do padrão estatístico esperado.  
Essas colunas servem para **inspeção e auditoria**, sem alterar nem remover dados — a decisão de tratamento posterior (ajuste, exclusão ou manutenção) é **de negócio**.

---

### ⚙️ Como funciona

1. **Verifica a configuração**
   - A execução depende da chave `detect_outliers` no arquivo `config/defaults.json`.  
     Se estiver definida como `true`, a detecção é realizada.

2. **Seleciona automaticamente as colunas elegíveis**
   - São analisadas apenas **colunas numéricas contínuas**, ou seja:
     - Tipo `int` ou `float`
     - Com mais de dois valores distintos (evitando variáveis binárias)
   - Exemplo: `tenure`, `MonthlyCharges`, `TotalCharges`.

3. **Escolhe o método estatístico**
   - `"iqr"` → usa o **Intervalo Interquartil (Interquartile Range)**:
     - Calcula Q1 (25º percentil) e Q3 (75º percentil)
     - Define limites:  
       Inferior = Q1 − 1.5 × IQR  
       Superior = Q3 + 1.5 × IQR
     - Valores fora desse intervalo são marcados como outliers.
   - `"zscore"` → usa o **desvio-padrão (Z-Score)**:
     - Calcula a média (μ) e o desvio-padrão (σ)
     - Para cada valor, obtém `z = (x − μ) / σ`
     - Valores com |z| > 3 são considerados outliers.

4. **Criação das colunas de flag**
   - Para cada variável analisada, é criada uma nova coluna:
     ```
     <coluna>_is_outlier
     ```
   - O valor será:
     - `True` → registro identificado como outlier  
     - `False` → registro dentro do intervalo esperado  
   - O total de flags criadas é exibido no log e registrado em `reports/outliers/summary.csv`.

5. **Registro no log**
   - Exemplo de log real:
     ```
     [apply_outlier_flags] flags criadas: 3 | método=iqr
     ```
     Isso significa que 3 colunas numéricas contínuas foram analisadas e receberam flags de outlier.

---

### 📊 Exemplo de resultado

| tenure | MonthlyCharges | TotalCharges | tenure_is_outlier | MonthlyCharges_is_outlier | TotalCharges_is_outlier |
|--------|----------------|--------------|-------------------|----------------------------|--------------------------|
| 5      | 35.50          | 190.00       | False             | False                      | False                    |
| 72     | 120.00         | 9999.99      | False             | **True**                   | **True**                 |

---

### 🧾 Relatórios e visualização

- O resumo de detecção é salvo em:  
  `reports/outliers/summary.csv`
- O notebook exibe cards com:
  - **Método e total de flags criadas**
  - **Colunas com outliers detectados**
  - **Percentual de registros afetados**
- Nenhum dado original é removido — apenas sinalizado.

---

> 💡 **Resumo:**  
> Esta célula aplica uma **detecção estatística de valores atípicos** nas colunas numéricas contínuas.  
> Os resultados são armazenados em colunas `*_is_outlier` e em relatórios dedicados, preservando a integridade do dataset para decisão de negócio posterior.


In [12]:
# ——— Outliers ———

cfg = ud.load_config()  # ou use seu dict manual
df2, out_info = ud.apply_outlier_flags(df, config=cfg)

# render organizado
ud.render_outlier_flags(out_info, df=df2, top_n=20)

# Se quiser seguir o pipeline com df já marcado:
df = df2


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:08,974 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:09,028 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:09,034 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:09,065 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [manifest] step='save_report_df' registrado.
2025-11-13 21:27:09,086 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo:

,Métrica,Valor
0,Linhas no df,7043
1,Flags criadas,5
2,Colunas com outliers (>0),1
3,Total de marcações de outlier,684


,outliers,pct
flag,,
charge_gap_is_outlier,684,9.71
tenure_is_outlier,0,0.00
MonthlyCharges_is_outlier,0,0.00
TotalCharges_is_outlier,0,0.00
charge_gap_log1p_is_outlier,0,0.00


## 🛠️ Engenharia de Atributos

Criação de novas colunas que capturem **relações, proporções, categorias ou padrões** relevantes ao negócio.  
Essa etapa é flexível e pode ser tanto automática (por regras padrão) quanto manual (definida via `config`).

---

### ⚙️ O que o bloco de código faz

- Verifica se `feature_engineering` está **habilitado no config**.  
- Aplica regras **automáticas seguras** (heurísticas Telco clássicas), como:
  - `avg_charge_per_month` → razão entre `TotalCharges / tenure`
  - `charge_gap` → diferença entre o valor esperado (`MonthlyCharges * tenure`) e o total pago
  - `is_long_tenure_24m` → flag para clientes com mais de 24 meses de contrato
- Executa transformações **condicionais à assimetria dos dados**, aplicando `log1p` em colunas numéricas com forte distorção (|skew| > 1).  
- Ignora colunas sem valor analítico para transformação (`*_was_missing`, binárias como `0/1`).  
- Permite **regras personalizadas** via `config`, como:
  - `ratios`: lista de pares `[numerador, denominador, nome_da_nova_coluna]`
  - `log1p_cols`: colunas nas quais aplicar transformação logarítmica
  - `binaries`: limiares para gerar flags booleanas
  - `date_parts`: colunas de data das quais extrair partes (ano, mês, semana...)

Quando nenhuma regra é aplicável, o notebook retorna:

```
ℹ️ Nenhuma engenharia de atributos aplicada.
   - Verifique config['feature_engineering'].
   - Exemplos de regras: log1p_cols, ratios, binaries, date_parts.
   - Habilite enable_default_rules para heurísticas automáticas.
```

---

### 💡 Exemplos

- **Razões e proporções:** `TotalCharges / tenure` → gasto médio mensal.  
- **Flags categóricas:** `Contract == 'Month-to-month'` → 1 se contrato mensal.  
- **Contagens de serviços:** soma de colunas binárias (Yes/No).  

Essas novas features tornam as análises mais ricas e **melhoram a performance de modelos preditivos**.

---

### 🧠 Boas práticas
- Use nomes **claros e descritivos** para novas features (`avg_charge_per_month`, `is_tenure_ge_12m`, etc.).  
- Evite criar colunas redundantes ou altamente correlacionadas.  
- Sempre documente cada feature gerada — especialmente se for usada em modelagem posterior.

---

> 💡 **Resumo:**  
> Esta célula adiciona inteligência ao dataset, criando variáveis derivadas que capturam padrões implícitos e aumentam o poder explicativo dos modelos,  
> combinando **heurísticas seguras automáticas** com **regras personalizáveis** via configuração.


In [13]:
# --- Engenharia de Atributos -------------------------------------------------
# Regras opcionais no config:
# config.get("feature_engineering") pode ser:
#   - True/False
#   - dict com opções:
#       {
#         "enable_default_rules": True,
#         "log1p_cols": ["MonthlyCharges"],
#         "ratios": [["TotalCharges","tenure","avg_charge_per_month"]],  # num/den->nome
#         "binaries": [["tenure", 24, "is_long_tenure"]],                # col, limiar, nome
#         "date_parts": ["order_date"]                                   # extrai ano/mes/dia/semana
#       }

# helpers de proteção
def _is_flag_col(c: str) -> bool:
    return isinstance(c, str) and c.endswith("_was_missing")

def _is_binary(s: pd.Series) -> bool:
    try:
        vals = pd.to_numeric(s, errors="coerce")
        nun = int(vals.dropna().nunique())
        return nun <= 2  # binária clássica (0/1 ou duas categorias)
    except Exception:
        return False

# ---------------------------------------------------------------------------
# 🔒 Helpers "NaN-safe" (apenas adicionados; não removem nada existente)
def _safe_div(num, den, fallback=0.0) -> pd.Series:
    """
    Divisão segura: retorna 'num/den' quando den>0 e num não-nulo; caso contrário usa 'fallback'.
    Mantém alinhamento pelo índice.
    """
    a = pd.to_numeric(num, errors="coerce")
    b = pd.to_numeric(den, errors="coerce")
    out = np.where((~pd.isna(a)) & (b.astype("float64") > 0), a / b, fallback)
    return pd.Series(out, index=a.index)

def _signed_log1p(x: pd.Series) -> pd.Series:
    """
    log1p assinado: sign(x) * log1p(|x|) — evita NaN de domínio para valores ≤ -1.
    Mantém NaN somente onde a entrada é NaN.
    """
    v = pd.to_numeric(x, errors="coerce")
    return np.sign(v) * np.log1p(np.abs(v))
# ---------------------------------------------------------------------------

fe_cfg = config.get("feature_engineering", False)
if not fe_cfg:
    print("⚙️ FE desabilitada em config['feature_engineering'].")
else:
    created = []

    # helper p/ log
    def _log(msg):
        try:
            logger.info(msg)  # se você tiver logger configurado
        except:
            print(msg)

    # Normaliza config em dict
    if isinstance(fe_cfg, bool):
        fe_cfg = {"enable_default_rules": True}

    # -------------------- 1) Regras explícitas do config --------------------
    # (a) Razões numéricas definidas pelo usuário: [[num, den, nome], ...]
    for rule in (fe_cfg.get("ratios", []) or []):
        if len(rule) == 3:
            num, den, out = rule
            if {num, den}.issubset(df.columns):
                # (ajuste NaN-safe)
                df[out] = _safe_div(df[num], df[den], fallback=0.0)
                created.append(out)

    # (b) Log1p em colunas positivas (config) — evita flags e binárias
    for col in (fe_cfg.get("log1p_cols", []) or []):
        if col in df.columns and not _is_flag_col(col) and not _is_binary(df[col]):
            s = pd.to_numeric(df[col], errors="coerce")
            pos = s > 0
            if pos.any():
                out = f"{col}_log1p"
                if out not in df.columns:  # não recriar
                    df[out] = np.nan
                    df.loc[pos, out] = np.log1p(s.loc[pos])
                    created.append(out)

    # (c) Binários por limiar: [[col, threshold, out_name], ...]
    for rule in (fe_cfg.get("binaries", []) or []):
        if len(rule) == 3:
            col, thr, out = rule
            if col in df.columns:
                try:
                    df[out] = (pd.to_numeric(df[col], errors="coerce") >= float(thr))
                    created.append(out)
                except Exception:
                    pass

    # (d) Partes de data (ano, mês, dia, semana ISO)
    for col in (fe_cfg.get("date_parts", []) or []):
        if col in df.columns and pd.api.types.is_datetime64_any_dtype(df[col]):
            for part in ["year", "month", "day", "week"]:
                out = f"{col}_{part}"
                try:
                    if part == "week":
                        df[out] = getattr(df[col].dt.isocalendar(), "week")
                    else:
                        df[out] = getattr(df[col].dt, part)
                    created.append(out)
                except Exception:
                    pass

    # -------------------- 2) Regras padrão (heurísticas seguras) ------------
    if fe_cfg.get("enable_default_rules", True):
        # (a) Telco clássico: cobrança média mensal estimada (NaN-safe para tenure==0)
        if {"TotalCharges", "tenure"}.issubset(df.columns):
            out = "avg_charge_per_month"
            df[out] = _safe_div(df["TotalCharges"], df["tenure"], fallback=0.0)
            created.append(out)

        # (b) Diferença entre o esperado (tenure*Monthly) e TotalCharges
        if {"MonthlyCharges", "tenure", "TotalCharges"}.issubset(df.columns):
            exp_total = (
                pd.to_numeric(df["MonthlyCharges"], errors="coerce") *
                pd.to_numeric(df["tenure"], errors="coerce")
            )
            out = "charge_gap"
            df[out] = pd.to_numeric(df["TotalCharges"], errors="coerce") - exp_total
            created.append(out)

            # (b.1) log1p do charge_gap — usa versão assinada para evitar NaN de domínio
            out_log = f"{out}_log1p"
            if out_log not in df.columns:
                df[out_log] = _signed_log1p(df[out])
                created.append(out_log)

        # (c) Limiar simples de fidelidade
        if "tenure" in df.columns:
            out = "is_long_tenure_24m"
            df[out] = (pd.to_numeric(df["tenure"], errors="coerce") >= 24)
            created.append(out)

        # (d) log1p automático em numéricas com valores > 0 e forte assimetria
        num_cols = df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            if _is_flag_col(col) or _is_binary(df[col]):
                continue
            s = pd.to_numeric(df[col], errors="coerce")
            if (s > 0).sum() > 0:
                skew = s.skew(skipna=True)
                if pd.notna(skew) and abs(skew) > 1.0:
                    out = f"{col}_log1p"
                    # evita recriar se já existe por config
                    if out not in df.columns:
                        df[out] = np.log1p(s.clip(lower=0))
                        created.append(out)

    # -------------------- 3) Relato do que foi feito -------------------------
    if created:
        created = sorted(set(created))
        _log(f"[feature_engineering] Features criadas: {len(created)} -> {created[:8]}{'...' if len(created)>8 else ''}")
        # Visual rápido (limita para não poluir)
        display(df[created].head(10))
    else:
        _log("[feature_engineering] Nenhuma feature criada (sem regras aplicáveis para as colunas atuais).")
        print(
            "ℹ️ Nenhuma engenharia de atributos aplicada.\n"
            "   - Verifique `config['feature_engineering']`.\n"
            "   - Exemplos de regras: "
            "`log1p_cols`, `ratios`, `binaries`, `date_parts`.\n"
            "   - Habilite `enable_default_rules` para heurísticas automáticas."
        )


2025-11-13 21:27:10,258 | INFO | [feature_engineering] Features criadas: 3 -> ['avg_charge_per_month', 'charge_gap', 'is_long_tenure_24m']


,avg_charge_per_month,charge_gap,is_long_tenure_24m
0,29.850000,0.00,False
1,55.573529,-46.80,True
2,54.075000,0.45,False
3,40.905556,-62.75,True
4,75.825000,10.25,False
5,102.562500,23.30,False
6,88.609091,-10.80,False
7,30.190000,4.40,False
8,108.787500,111.65,True
9,56.257258,6.65,True


# 📅 Tratamento de Datas

Esta etapa detecta e converte colunas de datas de forma **automática e controlada**, criando **features temporais** úteis para análises e modelos preditivos.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas de data

O sistema procura automaticamente colunas com nomes que contenham termos como  
`date`, `data`, `dt_`, `_dt`, `_date`, `_at`, `timestamp`, `created`, `updated`.

Também é possível **forçar colunas específicas** definindo manualmente em:

```python
date_cfg["explicit_cols"] = ["StartDate", "EndDate"]
```

---

### 🔹 Conversão para datetime com auditoria

Cada coluna candidata é testada com diferentes formatos e tentativas de *parsing*.  
O relatório **`parse_report`** mostra:

| column      | parsed_ratio | converted |
|--------------|--------------|------------|
| order_date   | 1.00         | True       |
| start_date   | 0.35         | False      |

- **column:** nome da coluna testada  
- **parsed_ratio:** porcentagem de valores convertidos com sucesso  
- **converted:** indica se a coluna foi convertida (baseado no `min_ratio`)

---

### 🔹 Criação automática de features temporais

Para cada coluna convertida em `datetime`, são geradas variáveis derivadas como:

`*_year`, `*_month`, `*_day`, `*_dayofweek`, `*_quarter`, `*_week`,  
`*_is_month_start`, `*_is_month_end`

O prefixo das novas colunas é o **nome original da coluna de data**.

---

### 🔎 Scanner de candidatos (modo silencioso)

Caso **nenhuma coluna de data seja detectada**, o notebook realiza uma varredura leve nas colunas de texto (`object`)  
para encontrar valores que *parecem* datas, exibindo o relatório:

| column | parse_ratio | sample_examples |
|---------|--------------|----------------|
| hire_dt | 0.42 | ['2020-01-01', '2021-05-13'] |

Esse scanner:
- roda de forma **silenciosa** (sem `UserWarning` do pandas);
- analisa apenas uma amostra dos dados (mais rápido);
- sugere colunas para incluir em `dates.explicit_cols` ou `dates.formats`.

---

### 🧱 Comportamento defensivo e visual limpo

- Se **nenhuma data for detectada**, a célula apenas informa:
  ```
  📅 Tratamento de Datas
  Nenhuma coluna de data detectada ou convertida.
  ```
- Se houver candidatos, eles são listados em uma tabela visual organizada.
- Se datas forem detectadas, as novas features são mostradas com cards e prévia de valores.

---

### ⚙️ Configuração (`date_cfg`)

| Parâmetro       | Descrição                                                        | Exemplo               |
|-----------------|------------------------------------------------------------------|------------------------|
| `detect_regex`  | Padrão para localizar colunas com nomes de data                 | `"date|timestamp|_at"` |
| `explicit_cols` | Lista manual de colunas a converter                             | `["StartDate"]`       |
| `dayfirst`      | Define se o formato é D/M/Y                                     | `True` para 🇧🇷        |
| `utc`           | Define se a conversão deve ser em UTC                           | `False`               |
| `formats`       | Lista de formatos específicos                                   | `["%d/%m/%Y"]`        |
| `min_ratio`     | Fração mínima de parsing bem-sucedido para aceitar a conversão  | `0.8`                 |

---

> 💡 **Resumo:**  
> Esta célula realiza o reconhecimento e parsing automático de colunas de data,
gera variáveis derivadas como ano, mês, dia e semana,
e, quando não há datas, executa um **scanner silencioso** para sugerir possíveis colunas.  
> Tudo isso é exibido em uma **interface limpa e defensiva**, sem gerar warnings,
mantendo o pipeline estável e transparente.


In [14]:
# --- Tratamento de Datas -----------------------------------------------------
date_cfg = dict(config.get("dates", {})) or {
    "detect_regex": r"(date|data|dt_|_dt$|_date$|_at$|time|timestamp|created|updated)",
    "explicit_cols": [],
    "dayfirst": False,
    "utc": False,
    "formats": [],
    "min_ratio": 0.80,
    "report_path": "date_parse_report.csv"
}

# 1) Parsing + relatório (mantém sua API atual)
df, parse_report, parsed_cols = ud.parse_dates_with_report_cfg(df, date_cfg)

# 2) Se não há colunas convertidas, roda um scanner SILENCIOSO e RÁPIDO
candidates = None
created = []
if not parsed_cols:
    candidates = ud.scan_date_candidates(
        df,
        cfg=date_cfg,
        min_ratio=0.20,   # coluna “suspeita” se >= 20% parseável
        sample=2000       # acelera e reduz barulho
    )
else:
    # 3) Se houve parsing, expande features de data (como você já fazia)
    created = ud.expand_date_features_plus(
        df, parsed_cols,
        features=["year","month","day","dayofweek","quarter","week","is_month_start","is_month_end"],
        prefix_mode="auto"
    )
    logger.info(f"[dates] features criadas: {len(created)}")

# 4) Render organizado (cards + tabelas)
ud.render_date_step(parsed_cols, parse_report=parse_report, candidates=candidates, created_features=created)

# 5) Se quiser um preview das features criadas:
if created:
    display(df[created].head(10))


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:11,218 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:11,225 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:11,256 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [manifest] step='save_report_df' registrado.
2025-11-13 21:27:11,274 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\date_parse_report.csv (0 linhas)
2025-11-13 21:27:11,277 | INFO | [report] salvo: C:\Users\fabio\Projetos DEV\data projects\telco-c

# 🗓️ Criação da Tabela Calendário (`dim_date`)

Esta etapa constrói automaticamente uma **tabela calendário completa** — também chamada de **dimensão de tempo** — a partir de uma coluna de data existente no dataset principal.  
Essa dimensão é essencial para **análises temporais**, **dashboards** e **modelos preditivos**, permitindo agrupamentos e comparações ao longo do tempo.

---

## 📋 O que acontece aqui

### 🔹 Detecção e validação da coluna de data

A função `run_calendar_step()` tenta identificar automaticamente uma coluna de data no DataFrame.  
Se nenhuma for encontrada, a etapa é **pulada com segurança**, exibindo o card:

```
📆 Dimensão Calendário
Status: skipped
Nenhum campo de data disponível. Defina 'date_col' ou trate datas antes.
```

Você também pode definir manualmente a coluna e a frequência de agregação:

```python
CAL_DATE_COL = "order_date"   # nome da coluna datetime
CAL_FREQ = "D"                # frequência: "D" (diário), "W" (semanal), "M" (mensal)
CAL_OUT = "data/processed/dim_date.parquet"
```

---

### 🔹 Construção da dimensão calendário

Quando uma coluna válida é encontrada, a função:

1. **Converte automaticamente para datetime**, se necessário (quando a taxa de parsing ≥ 80%).  
2. **Gera todas as datas** entre o mínimo e o máximo da coluna informada.  
3. Cria variáveis derivadas como:

| Coluna            | Descrição                                |
|--------------------|-------------------------------------------|
| `date`             | Data base (chave primária)               |
| `year`             | Ano                                      |
| `month`            | Mês numérico                             |
| `day`              | Dia do mês                               |
| `quarter`          | Trimestre (1–4)                          |
| `week`             | Semana ISO                               |
| `dayofweek`        | Dia da semana (0 = segunda, 6 = domingo) |
| `is_month_start`   | Indica se a data é o primeiro dia do mês |
| `is_month_end`     | Indica se a data é o último dia do mês   |

---

### 🔹 Armazenamento e registro

A tabela é salva automaticamente pelo `ud.save_table()`  
→ o caminho padrão é `data/processed/dim_date.parquet` (ou `.csv`, se preferir).  

Se o catálogo `T` estiver ativo, a função também registra a tabela:

```python
T.add("dim_date", dim_date)
```

Assim, ela pode ser reutilizada em outras partes do pipeline.

---

### ⚙️ Execução simplificada

O novo fluxo exige apenas uma chamada direta:

```python
info = ud.run_calendar_step(
    df,
    date_col=CAL_DATE_COL,
    freq=CAL_FREQ,
    output=CAL_OUT,
    config=config,
    paths=paths,
    catalog=T if 'T' in globals() else None
)

ud.render_calendar_step(info)
```

O `render_calendar_step()` exibe um resumo visual com:
- Status da execução (✅ ok / ⚠️ skipped)
- Coluna usada
- Frequência
- Período coberto
- Caminho de saída
- Prévia das primeiras linhas da tabela

---

### 🧠 Comportamento defensivo

- Se nenhuma coluna de data existir → o pipeline **não quebra**.  
- Se a coluna estiver fora do formato datetime → o sistema **sugere executar o tratamento de datas**.  
- Se tudo estiver correto → a tabela é criada, salva e registrada automaticamente.

---

> 💡 **Resumo:**  
> A função `run_calendar_step()` cria uma **tabela calendário completa e reutilizável**, pronta para junções e agregações temporais.  
> O processo é **automatizado**, **seguro** e segue as convenções do template (`data/processed/`).  
> Caso não haja colunas de data, a etapa é apenas **pulada com aviso elegante**, mantendo o pipeline fluido e robusto.


In [15]:
# --- Criação da Tabela Calendário (dim_date) --------------------------------
# (opcional) definir manualmente; se None, a função tenta descobrir:
CAL_DATE_COL = None            # ex.: "order_date"  | None → auto-descoberta (primeira datetime)
CAL_FREQ     = "D"             # "D", "MS", "QS", etc.
CAL_OUT      = None            # ex.: "data/processed/dim_date.parquet" | None → default do template

info = ud.run_calendar_step(
    df,
    date_col=CAL_DATE_COL,
    freq=CAL_FREQ,
    output=CAL_OUT,
    config=config,
    paths=paths,    # se você tem o objeto 'paths' no notebook
    catalog=T if 'T' in globals() else None  # registra no catálogo se existir
)

ud.render_calendar_step(info)


[INFO] [calendar] Nenhum campo de data disponível. Defina 'date_col' ou trate datas antes.
2025-11-13 21:27:12,432 | INFO | [calendar] Nenhum campo de data disponível. Defina 'date_col' ou trate datas antes.


# 📝 Tratamento de Texto (opcional)

Esta etapa extrai **métricas numéricas e lógicas a partir de colunas textuais**, transformando texto livre em informações quantitativas úteis para **análise exploratória e modelagem**.  

É uma forma leve e controlada de **estruturar dados não numéricos**, sem recorrer a técnicas avançadas de NLP.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas textuais

O sistema identifica automaticamente todas as colunas com tipo `object` ou `string`,  
ignorando as listadas na *blacklist* (`TEXT_CFG["blacklist"]`).

Essas colunas geralmente representam textos curtos, como categorias, descrições,  
nomes de serviços ou observações registradas em texto livre.

---

### 🔹 Limpeza e padronização

Antes de calcular as métricas, os textos passam por uma limpeza leve:
- Remoção de **espaços duplicados** e **trim** nas extremidades.  
- Conversão para **minúsculas**, garantindo consistência em análises de termos.

Esses comportamentos são controlados pelas chaves:

```python
"lower": True,
"strip_collapse_ws": True
```

---

### 🔹 Geração de métricas textuais

Para cada coluna textual analisada, o sistema calcula estatísticas básicas e as salva em um **resumo CSV**:

| Campo do Resumo  | Descrição                                      |
|------------------|------------------------------------------------|
| `column`         | Nome da coluna textual analisada               |
| `non_null`       | Quantidade de células não nulas                |
| `avg_len`        | Comprimento médio (número de caracteres)       |
| `avg_words`      | Quantidade média de palavras                   |

Essas métricas são úteis para identificar colunas com textos longos, inconsistentes ou padronizados demais.

---

### 🔹 Presença de palavras-chave

O sistema também verifica a ocorrência de palavras específicas em cada coluna textual.

Exemplo:

```python
TEXT_CFG["keywords"] = ["error", "cancel", "premium"]
```

Cria métricas como:
- `kw_error_count`  
- `kw_cancel_count`  
- `kw_premium_count`

Essas colunas representam **quantas vezes cada termo aparece** no conjunto textual — e são úteis em análises de sentimento, churn, reclamações ou feedbacks de clientes.

---

### 🔹 Exportação e visualização

O resultado é salvo automaticamente em:

```
reports/artifacts/text_features/text_features_summary.csv
```

E o notebook exibe um **painel visual resumido**, via função:

```python
ud.render_text_features_summary(text_summary)
```

Esse painel apresenta:
- Total de colunas textuais analisadas  
- Quantidade total de células não nulas  
- Top colunas por comprimento médio  
- Ocorrências agregadas por palavra-chave  

---

### ⚙️ Configuração (`TEXT_CFG`)

| Parâmetro              | Descrição                                                       | Exemplo                              |
|------------------------|------------------------------------------------------------------|--------------------------------------|
| `lower`                | Converte o texto para minúsculas                                | `True`                               |
| `strip_collapse_ws`    | Remove espaços duplicados e limpa extremidades                  | `True`                               |
| `keywords`             | Lista de termos a detectar no texto                             | `["error", "cancel", "premium"]`     |
| `blacklist`            | Colunas a ignorar durante o processamento                       | `["customerID"]`                     |
| `export_summary`       | Salva relatório com resumo das features geradas                 | `True`                               |

---

### 🧠 Boas práticas

- Aplique esta etapa apenas em colunas realmente textuais (não IDs ou códigos).  
- Personalize a lista de **palavras-chave** conforme o contexto do dataset.  
- Caso o dataset não possua colunas de texto, o processo será ignorado com segurança.  
- Utilize o painel e o arquivo `text_features_summary.csv` para auditoria e documentação.

---

> 💡 **Resumo:**  
> Esta etapa transforma colunas textuais em **indicadores numéricos e lógicos**, calculando comprimento médio, contagem de palavras e ocorrência de termos específicos.  
> Os resultados são exportados em CSV e apresentados de forma visual no notebook,  
> garantindo **clareza, rastreabilidade e integração consistente** dos dados textuais ao pipeline.


In [16]:
# ——— Tratamento de Texto (com visual limpo) ———
TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    summary_path = ud.get_artifacts_dir("text_features")

    # mantém sua função atual
    df, text_summary = ud.extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=summary_path
    )

    # nova renderização amigável
    ud.render_text_features_summary(
        text_summary,
        keywords=TEXT_CFG["keywords"],
        top=20
    )


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:14,283 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\text_features
2025-11-13 21:27:14,286 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\text_features
[INFO] [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\text_features\text_features_summary.csv (16 colunas)
2025-11-13 21:27:14,947 | INFO | [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\text_features\text_features_summary.csv (16 colunas)


C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\utils\utils_data.py:2115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[kw_flag] = s.str.contains(rf"\b{kw}\b", regex=True, na=False)


,Métrica,Valor
0,Colunas textuais analisadas,16
1,Total de células não nulas (texto),112688


,column,non_null,avg_len,avg_words
0,PaymentMethod,7043,18.57,2.44
1,Contract,7043,11.30,1.45
2,InternetService,7043,6.30,1.44
3,gender,7043,4.99,1.00
4,PhoneService,7043,2.90,1.00
5,PaperlessBilling,7043,2.59,1.00
6,Partner,7043,2.48,1.00
7,MultipleLines,7043,2.42,1.00
8,StreamingMovies,7043,2.39,1.00
9,StreamingTV,7043,2.38,1.00


,keyword_metric,ocorrencias
0,kw_error_count,0
1,kw_cancel_count,0
2,kw_premium_count,0


# 🎯 Criação e Validação da Variável-Alvo (`target`)

Esta etapa garante que o dataset contenha uma **variável-alvo binária** adequada para **modelos supervisionados de Machine Learning** — como classificação de *churn*, fraude, aprovação de crédito, etc.  

O processo segue o padrão definido no arquivo `config/defaults.json`, permitindo que a criação e validação da *target* sejam **automáticas, reproduzíveis e rastreáveis**.

---

## ⚙️ Configuração do Target

O comportamento é controlado pela seção `"target"` do arquivo de configuração.  
Exemplo típico (como no dataset *Telco Customer Churn*):

```json
"target": {
  "name": "Churn",
  "source": "Churn",
  "positive": "Yes",
  "negative": "No"
}
```

### Parâmetros
| Chave | Descrição | Exemplo |
|--------|------------|----------|
| `name` | Nome final da coluna de destino | `"Churn"` |
| `source` | Coluna de origem (caso o nome seja diferente) | `"Churn"` |
| `positive` | Valor que representa a classe positiva | `"Yes"` |
| `negative` | Valor que representa a classe negativa | `"No"` |

> 💡 **Dica:** se o dataset já tiver a coluna-alvo, o sistema apenas validará e manterá como está — sem sobrescrever.

---

### ⚙️ Execução e Lógica Interna

```python
df, target_name, class_map, tgt_report = ud.ensure_target_from_config(df, config, verbose=True)
display(tgt_report)
globals()["class_map"] = class_map
```

Etapas internas:

1. **Verifica se a coluna-alvo já existe**
   - Se existir (`Churn`), exibe mensagem:  
     ```
     [target] Coluna 'Churn' já existe — nenhuma ação necessária.
     ```
   - Caso contrário, cria automaticamente a coluna com base nas configurações.

2. **Aplica o mapeamento definido**
   - Exemplo: `Yes → 1`, `No → 0`.

3. **Gera um mini-relatório**
   - Inclui nome, status, origem e classes positivas/negativas.  
   - Retorna também o `class_map`, usado nos notebooks seguintes.

---

## 📊 Mini-Relatório de Validação

| target | status     | source | positive | negative |
|---------|-------------|---------|-----------|-----------|
| Churn   | já existe  | Churn   | Yes       | No        |

Esse relatório resume como o *target* foi identificado ou criado, garantindo **transparência e rastreabilidade**.

---

## 🔍 Verificação de Consistência

Após validar ou criar o *target*, o notebook checa os valores e tipos:

```python
print(df["Churn"].unique())
print(df["Churn"].value_counts())
```

Resultados esperados:

| Caso | Exemplo de Saída | Interpretação |
|------|------------------|---------------|
| Target binário correto | `[0, 1]` | ✅ Pronto para ML |
| Target textual | `['Yes', 'No']` | 🔄 Converter com `.map({'Yes':1,'No':0})` |
| Target booleano | `[True, False]` | 🔄 Converter com `.astype(int)` |

---

## 🧠 Boas Práticas

- Defina claramente qual valor representa a **classe positiva** (ex.: `"Yes"`, `"Exited"`, `"Churned"`).  
- Evite múltiplas colunas redundantes — centralize em uma só (`Churn`).  
- Para modelos multiclasse, amplie o mapeamento conforme necessário.  
- Sempre valide a distribuição de classes com `value_counts()` antes da modelagem.

---

## 🚀 Integração com o N2 (Modelagem)

A variável `class_map` é registrada globalmente para que o **Notebook N2** possa reutilizá-la:

```python
globals()["class_map"] = class_map
```

Isso garante que o mesmo mapeamento seja aplicado nas fases de:
- Conversão de rótulos (`1 → 'Yes'`, `0 → 'No'`);  
- Exibição de métricas e resultados em termos legíveis.

---

✅ **Status esperado após execução**
- Target identificado: `Churn`  
- Tipo: categórico binário (`Yes` / `No`)  
- Proporção positiva: ~26 %  
- Ação: validado e mantido (nenhuma recriação necessária)

---

> 💡 **Resumo:**  
> Esta célula assegura que o dataset possua uma variável-alvo binária consistente e configurada de forma reprodutível.  
> Se a coluna já existir, é apenas validada; se não, é criada de acordo com o `config`.  
> O processo registra o mapeamento global (`class_map`) para uso direto nas próximas etapas do pipeline (N2).  


In [17]:
# === 🎯 Criação e Validação da Variável-Alvo (target) =========================
import importlib, utils.utils_data as ud
importlib.reload(ud)

# 1️⃣ — Configuração base do target --------------------------------------------
# Define manualmente o nome da coluna e o mapeamento de classes
config.setdefault("target", {})
config["target"].update({
    "name": "Churn",       # Nome da variável-alvo final
    "source": "Churn",     # Coluna de origem (já presente no df)
    "positive": "yes",     # Classe positiva (ajuste conforme o dataset)
    "negative": "no"       # Classe negativa
})

# 2️⃣ — Execução e relatório ----------------------------------------------------
tgt_info = ud.fix_target_then_summary(df, config, verbose=True)
ud.render_target_summary(tgt_info)

# Atualiza o DataFrame (caso tenha sido ajustado internamente)
df = tgt_info["df"]

# 3️⃣ — Diagnóstico rápido (opcional) -----------------------------------------
from IPython.display import display
print("\n🔍 Verificando a consistência do tipo e dos valores:")
if config["target"]["name"] in df.columns:
    target_col = config["target"]["name"]
    display(df[target_col].value_counts(dropna=False).to_frame("contagem"))
else:
    print(f"⚠️ A coluna '{config['target']['name']}' ainda não existe no DataFrame.")


[target] Coluna 'Churn' já existe — nenhuma ação necessária.
[target] Coluna 'Churn' já existe — nenhuma ação necessária.


,classe,contagem,percentual
0,no,5174,73.46%
1,yes,1869,26.54%
2,TOTAL,7043,100.00%



🔍 Verificando a consistência do tipo e dos valores:


,contagem
Churn,
no,5174
yes,1869


# 🔢 Codificações Categóricas & Escalonamento Numérico

Esta etapa converte variáveis **categóricas** em representações numéricas e **escalona** variáveis **numéricas** para estabilizar modelos.  
Ela garante que o dataset esteja **padronizado e pronto para a modelagem (N2)**, além de registrar os **artefatos reutilizáveis** de transformação.

---

## 📋 O que acontece aqui

1. **Codificação de variáveis categóricas**
   - Detecta colunas `object`, `string` ou `category` (já padronizadas).
   - Estratégias controladas por `config["encoding"]`:
     - `"onehot"` → cria colunas dummies (`0/1`) — ideal para modelos lineares e árvores modernas.
     - `"label"` → converte categorias em inteiros — útil para árvores de decisão (atenção: ordem não tem significado).
   - Os **encoders** gerados são salvos para reuso posterior no N2.

2. **Escalonamento de variáveis numéricas**
   - Identifica colunas numéricas (incluindo novas features derivadas).
   - Estratégias definidas em `config["scaling"]`:
     - `"standard"` → aplica Z-score (média 0, desvio padrão 1).
     - `"minmax"` → normaliza no intervalo [0, 1].
   - Os **scalers** são salvos para garantir consistência entre treino e predição.

3. **Relatórios e auditoria**
   - Gera um **resumo tabular** com as colunas codificadas e escalonadas.
   - Artefatos salvos automaticamente em:
     ```
     reports/artifacts/encoders/
     reports/artifacts/scalers/
     ```

---

## ⚙️ Configuração esperada (exemplo)

```python
config["encoding"] = {
  "enabled": True,
  "strategy": "onehot",       # "onehot" | "label"
  "drop_first": False,        # opcional para onehot
  "exclude": ["customerID"]   # nunca codificar IDs
}

config["scaling"] = {
  "enabled": True,
  "strategy": "standard",     # "standard" | "minmax"
  "exclude": []               # ex.: colunas já normalizadas
}
```

---

## 🧠 Boas práticas

- **Padronize categorias** antes de codificar (vide etapa de *Padronização Categórica*).  
- **Evite codificar chaves técnicas** ou identificadores únicos (como `customerID`).  
- **Salve e reaplique** os mesmos encoders/scalers no N2 para consistência nos modelos.  
- Se houver muitas categorias raras, realize **agregação ou limpeza** prévia.  
- Para reduzir dimensionalidade, use `drop_first=True` no One-Hot (elimina redundância).

---

> 💡 **Resumo:**  
> Esta célula transforma variáveis categóricas e numéricas em formatos compatíveis com modelos estatísticos e de machine learning.  
> Os artefatos gerados (encoders e scalers) são **registrados, versionados e reutilizáveis**, assegurando **reprodutibilidade e integridade do pipeline de dados**.


In [19]:
# --- 🔢 Codificação Categórica & Escalonamento Numérico (preservando o target) ---

import copy
import utils.utils_data as ud

# 0) Descobre/valida o nome do alvo
tgt_name = (config.get("target", {}) or {}).get("name") or config.get("target_column") or "Churn"
if tgt_name not in df.columns:
    raise KeyError(f"Target '{tgt_name}' não encontrado no df. Colunas: {list(df.columns)[:10]}...")

# 1) Separa X e y para não deixar o alvo entrar na codificação/escalonamento
y = df[tgt_name].copy()
X = df.drop(columns=[tgt_name])

# 2) Garante exclusões também via config (caso a função use internamente)
cfg2 = copy.deepcopy(config)
# Normaliza chaves novas
enc = cfg2.setdefault("encoding", {})
scl = cfg2.setdefault("scaling", {})

# Habilita pela config nova (se estiver usando o esquema antigo)
enc.setdefault("enabled", bool(cfg2.get("encode_categoricals", True)))
enc.setdefault("strategy", cfg2.get("encoding_type", "onehot"))
enc.setdefault("exclude", [])
scl.setdefault("enabled", bool(cfg2.get("scale_numeric", False)))
scl.setdefault("strategy", cfg2.get("scaler", "standard"))
scl.setdefault("exclude", [])

# Garante que o target NUNCA seja codificado/escalonado
if tgt_name not in enc["exclude"]:
    enc["exclude"].append(tgt_name)
if tgt_name not in scl["exclude"]:
    scl["exclude"].append(tgt_name)

# 3) Executa a etapa unificada APENAS em X
X_enc, enc_scal_info = ud.run_encoding_and_scaling(X, cfg2)

# 4) Reanexa o alvo exatamente como estava (ordem de colunas: alvo por último ou primeiro, você escolhe)
df = X_enc.copy()
df[tgt_name] = y.values  # preserva dtype e valores originais

# 5) Renderização e verificação
ud.render_encoding_and_scaling(enc_scal_info)

print("\n🔒 Verificação do target após codificação:")
print("→ Target presente?", tgt_name in df.columns)
try:
    print(df[tgt_name].value_counts())
except Exception:
    print("Não foi possível exibir value_counts do target (verifique dtype).")



🔒 Verificação do target após codificação:
→ Target presente? True
Churn
no     5174
yes    1869
Name: count, dtype: int64


# 📦 Exportação de Artefatos (N1 → N2)

Esta etapa consolida e exporta os **artefatos finais do notebook N1** — garantindo que os dados tratados, os metadados e o manifesto de execução estejam disponíveis para o **N2 (Modelagem)**.  

É o fechamento da fase de preparação dos dados, onde tudo que foi limpo, transformado e enriquecido é salvo em **arquivos persistentes e reprodutíveis**.

---

## 🧭 Visão Geral do Processo

1. **Exporta os dados tratados** (`interim` e `processed`);
2. **Gera um arquivo de metadados (`meta.json`)** com a estrutura do dataset;
3. **Cria um manifesto (`manifest.json`)** com o resumo técnico da execução — memória usada, flags criadas, caminhos de saída, etc.

Tudo é salvo em caminhos padronizados e definidos dinamicamente a partir do `config`.

---

## ⚙️ Configurações e Caminhos Padrão

Os caminhos de saída podem ser definidos no `config/defaults.json`, ou usam valores padrão se ausentes:

| Chave | Caminho padrão | Descrição |
|--------|----------------|-----------|
| `data_interim_file` | `data/interim/interim.parquet` | Dados intermediários tratados |
| `data_processed_file` | `data/processed/processed.parquet` | Dados finais prontos para modelagem |
| `meta_file` | `artifacts/metadata/dataset_meta.json` | Metadados do dataset |
| `random_seed` | `42` (padrão) | Semente para reprodutibilidade |

A função `get_artifacts_dir("export")` cria automaticamente o diretório padrão de exportação:

```
reports/artifacts/export/
```

---

## 📋 Etapas principais do código

### 1️⃣ Exportação dos DataFrames

```python
OUTPUT_PROCESSED = Path(config.get("data_processed_file", "data/processed/processed.parquet"))
OUTPUT_INTERIM   = Path(config.get("data_interim_file",   "data/interim/interim.parquet"))

if config.get("export_interim", True):
    _save_df(df, OUTPUT_INTERIM)

if config.get("export_processed", True):
    _save_df(df, OUTPUT_PROCESSED)
```

A função `_save_df()` detecta automaticamente o formato pelo sufixo do arquivo:

| Extensão | Função utilizada | Compressão |
|-----------|------------------|-------------|
| `.parquet` | `to_parquet()` | Compactado (`snappy`) |
| `.csv` | `to_csv()` | UTF-8 |
| `.xlsx` | `to_excel()` | Excel |

---

### 2️⃣ Geração dos Metadados (`meta.json`)

Cria um registro detalhado da estrutura do dataset exportado.

```python
meta = {
  "dataset_name": config.get("dataset_name", "Dataset"),
  "version": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
  "target": target_col,
  "class_map": class_map,
  "columns": {
    "numeric": numeric_cols,
    "categorical": categorical_cols,
    "boolean": boolean_cols,
    "ignored": ignored_cols,
    "all": all_cols
  }
}
META_FILE.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")
```

Esse arquivo é essencial para o N2, pois indica:
- quais colunas são numéricas, categóricas e booleanas;
- qual é o `target`;
- quais colunas foram ignoradas;
- o mapeamento de classes (`class_map`, se existir);
- a versão e data de exportação.

> 💡 **Dica:**  
> você pode adicionar `meta["dtypes"] = {c: str(t) for c, t in df.dtypes.items()}`  
> para registrar os tipos exatos de cada coluna no arquivo de metadados.

---

### 3️⃣ Registro do Manifesto de Execução (`manifest.json`)

O manifesto é um **resumo técnico** da sessão atual — útil para auditoria e rastreabilidade.

```python
manifest = {
  "created_at": datetime.now().isoformat(timespec="seconds"),
  "random_seed": RANDOM_SEED,
  "config": config,
  "memory_mb": float(df.memory_usage(deep=True).sum() / (1024**2)),
  "outlier_flags": [c for c in df.columns if c.endswith("_is_outlier")],
  "imputed_flags": [c for c in df.columns if c.startswith("was_imputed_")],
  "shape": tuple(df.shape),
  "exported": {
    "interim": str(OUTPUT_INTERIM),
    "processed": str(OUTPUT_PROCESSED),
    "meta_file": str(META_FILE)
  }
}

(ARTIFACTS_DIR / "manifest.json").write_text(json.dumps(manifest, indent=2, ensure_ascii=False))
```

Campos mais importantes:
| Campo | Descrição |
|--------|------------|
| `created_at` | Data e hora da execução |
| `random_seed` | Semente aleatória utilizada |
| `memory_mb` | Tamanho em memória do DataFrame |
| `outlier_flags` / `imputed_flags` | Colunas criadas por outliers ou imputações |
| `shape` | Dimensão final do dataset |
| `exported` | Caminhos de saída dos artefatos gerados |

---

### 4️⃣ Log de conclusão

Ao final da célula, é exibido no notebook:

```
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export
Arquivos gerados:
- INTERIM:   data\interim\interim.parquet
- PROCESSED: data\processed\processed.parquet
- META:      artifacts\metadata\dataset_meta.json
```

---

## 🧠 Boas práticas

- **Versão dos dados**: use o campo `"version"` do meta para marcar reprocessamentos.
- **Controle de reprodutibilidade**: defina `random_seed` no config.
- **Organização**: mantenha sempre `interim`, `processed` e `meta` sob versionamento no Git.
- **Auditoria**: o `manifest.json` é a “assinatura digital” da execução do N1.
- **Portabilidade**: todo o caminho é relativo à raiz do projeto, garantindo compatibilidade com outros ambientes.

---

> 💡 **Resumo:**  
> Esta célula encerra o **notebook N1**, gerando todos os artefatos necessários para as etapas seguintes (EDA, modelagem e visualização).  
> Ela garante **reprodutibilidade**, **rastreamento** e **organização** dos resultados, consolidando as saídas do pipeline em formatos padronizados e facilmente reutilizáveis.


In [18]:
# =============================================================================
# 📦 Exportação de Artefatos (N1 → N2) — corrigindo os caminhos
# =============================================================================
from datetime import datetime
from pathlib import Path
import json
import numpy as np
import pandas as pd

PROJECT_ROOT = ud.ensure_project_root()
logger.info(f"[path] PROJECT_ROOT -> {PROJECT_ROOT}")

# 1) Caminhos corretos: diretório + arquivo
proc_dir = PROJECT_ROOT / Path(config.get("data_processed_dir", "data/processed"))
proc_file = config.get("data_processed_file", "processed.parquet")
OUTPUT_PROCESSED = proc_dir / proc_file

inter_dir = PROJECT_ROOT / Path(config.get("data_interim_dir", "data/interim"))
inter_file = config.get("data_interim_file", "interim.parquet")
OUTPUT_INTERIM = inter_dir / inter_file

META_FILE = PROJECT_ROOT / Path(config.get("meta_file", "artifacts/metadata/dataset_meta.json"))

# 2) Garante diretórios
OUTPUT_PROCESSED.parent.mkdir(parents=True, exist_ok=True)
OUTPUT_INTERIM.parent.mkdir(parents=True, exist_ok=True)
META_FILE.parent.mkdir(parents=True, exist_ok=True)

# 3) Salva tabelas respeitando a extensão
def _save_df(df_, path_: Path):
    ext = path_.suffix.lower()
    if ext == ".parquet":
        df_.to_parquet(path_, index=False)
    elif ext == ".csv":
        df_.to_csv(path_, index=False, encoding="utf-8")
    elif ext == ".xlsx":
        df_.to_excel(path_, index=False)
    else:
        raise ValueError(f"Extensão não suportada: {ext}")

if config.get("export_interim", True):
    _save_df(df, OUTPUT_INTERIM)

if config.get("export_processed", True):
    _save_df(df, OUTPUT_PROCESSED)

# 4) Metadados (igual ao seu, mantido)
target_cfg  = config.get("target", {}) or {}
target_col  = target_cfg.get("name", config.get("target_column", "target"))

all_cols = df.columns.tolist()
ignored_cols = config.get("ignored_columns", [])
candidate_features = [c for c in all_cols if c not in ignored_cols and c != target_col]

numeric_cols     = df[candidate_features].select_dtypes(include=[np.number]).columns.tolist()
boolean_cols     = df[candidate_features].select_dtypes(include=["bool", "boolean"]).columns.tolist()
categorical_cols = [c for c in candidate_features if c not in numeric_cols]

class_map = globals().get("class_map", None)
meta = {
    "dataset_name": config.get("dataset_name", "Dataset"),
    "version":      datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "target":       target_col,
    "class_map":    class_map,
    "columns": {
        "numeric":     numeric_cols,
        "categorical": categorical_cols,
        "boolean":     boolean_cols,
        "ignored":     ignored_cols,
        "all":         all_cols
    },
    "dtypes": {c: str(t) for c, t in df.dtypes.items()},
    "rows":   int(len(df)),
}
META_FILE.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("\n" + "="*80)
print("📦  EXPORTAÇÃO DE ARTEFATOS — RESUMO FINAL")
print("="*80)
print(f"🗂️  Dataset:        {meta['dataset_name']}")
print(f"🎯  Target:         {meta['target']}")
print(f"🧮  Linhas:         {meta['rows']:,}")
print(f"🔢  Numéricas:      {len(meta['columns']['numeric'])} colunas")
print(f"🔠  Categóricas:    {len(meta['columns']['categorical'])} colunas")
print(f"🔘  Booleanas:      {len(meta['columns']['boolean'])} colunas")
print(f"🧾  Ignoradas:      {len(meta['columns']['ignored'])} colunas")
print(f"🛠️  INTERIM salvo:  {OUTPUT_INTERIM}")
print(f"🛠️  PROCESSED salvo:{OUTPUT_PROCESSED}")
print("="*80 + "\n")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:21,112 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:21,115 | INFO | [path] PROJECT_ROOT -> C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis

📦  EXPORTAÇÃO DE ARTEFATOS — RESUMO FINAL
🗂️  Dataset:        Dataset
🎯  Target:         Churn
🧮  Linhas:         7,043
🔢  Numéricas:      62 colunas
🔠  Categóricas:    70 colunas
🔘  Booleanas:      54 colunas
🧾  Ignoradas:      0 colunas
🛠️  INTERIM salvo:  C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\data\interim\interim.parquet
🛠️  PROCESSED salvo:C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\data\processed\processed.parquet



# 🧭 Resumo final de sanidade

In [19]:
# =============================================================================
# ✅ N1 — Resumo final de sanidade (opcional e enxuto)
# =============================================================================
from pathlib import Path
import json
import numpy as np
import pandas as pd

if (config.get("reporting", {}).get("final_summary", True)):
    PROJECT_ROOT = ud.ensure_project_root()

    # Caminhos (alinhados ao bloco de exportação)
    OUTPUT_PROCESSED = PROJECT_ROOT / Path(config.get("data_processed_file", "data/processed/processed.parquet"))
    OUTPUT_INTERIM   = PROJECT_ROOT / Path(config.get("data_interim_file",   "data/interim/interim.parquet"))
    META_FILE        = PROJECT_ROOT / Path(config.get("meta_file",           "artifacts/metadata/dataset_meta.json"))
    MANIFEST_FILE    = ud.get_artifacts_dir("export") / "manifest.json"

    # Tipagem consolidada
    dtypes = df.dtypes.to_dict()
    num_cols     = [c for c,t in dtypes.items() if pd.api.types.is_numeric_dtype(t)]
    bool_cols    = [c for c,t in dtypes.items() if pd.api.types.is_bool_dtype(t)]
    cat_cols     = [c for c,t in dtypes.items() if (pd.api.types.is_string_dtype(t) or pd.api.types.is_categorical_dtype(t))]

    # Target (do config)
    target_col = (config.get("target", {}) or {}).get("name", config.get("target_column", "target"))
    target_ok  = target_col in df.columns

    # Artefatos existentes
    exists_processed = OUTPUT_PROCESSED.exists()
    exists_interim   = OUTPUT_INTERIM.exists()
    exists_meta      = META_FILE.exists()
    exists_manifest  = MANIFEST_FILE.exists()

    # Memória (MB)
    mem_mb = float(df.memory_usage(deep=True).sum()) / (1024**2)

    line = "="*80
    print("\n" + line)
    print("✅  N1 CONCLUÍDO — RESUMO DE SANIDADE")
    print(line)
    print(f"📐 Shape final do df:   {df.shape}")
    print(f"🎯 Target:              {target_col} ({'ok' if target_ok else 'ausente'})")
    print(f"🧮 Linhas registradas:  {len(df):,}")
    print(f"🧾 Colunas (totais):    {df.shape[1]}")
    print(f"   ├─ Numéricas:        {len(num_cols)}")
    print(f"   ├─ Categóricas:      {len(cat_cols)}")
    print(f"   └─ Booleanas:        {len(bool_cols)}\n")
    print("📦 Artefatos gerados:")
    print(f"   ├─ INTERIM:          {str(exists_interim):5} | {OUTPUT_INTERIM}")
    print(f"   ├─ PROCESSED:        {str(exists_processed):5} | {OUTPUT_PROCESSED}")
    print(f"   ├─ META:             {str(exists_meta):5} | {META_FILE}")
    print(f"   └─ MANIFEST:         {str(exists_manifest):5} | {MANIFEST_FILE}\n")
    print(f"🧠 Memória do DF:       {mem_mb:.2f} MB")
    print(line + "\n")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:21,933 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
2025-11-13 21:27:21,936 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\export
2025-11-13 21:27:21,943 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\telco-customer-churn-analysis\reports\artifacts\export

✅  N1 CONCLUÍDO — RESUMO DE SANIDADE
📐 Shape final do df:   (7043, 133)
🎯 Target:              Churn (ok)
🧮 Linhas registradas:  7,043
🧾 Colunas (totais):    133
   ├─ Numéricas:        116
   ├─ Categóricas:      17
   └─ Booleanas:        54

📦 Artefatos gerados:
   ├─ INTERIM:          Tru

C:\Users\fabio\AppData\Local\Temp\ipykernel_24864\3075353676.py:22: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols     = [c for c,t in dtypes.items() if (pd.api.types.is_string_dtype(t) or pd.api.types.is_categorical_dtype(t))]


## ✅ Checkpoint

- **Revisar** as colunas derivadas e decisões (imputação, outliers, codificação).  
- **Documentar** no README as escolhas de negócio e justificativas.  

## 📎 Anotações

Esta seção pode ser usada como bloco livre para observações específicas do projeto.


In [20]:
df.isna().sum().sort_values(ascending=False).head(10), df.isna().sum().sum()


(customerID                      0
 OnlineSecurity_word_count       0
 TechSupport_len                 0
 DeviceProtection_has_premium    0
 DeviceProtection_has_cancel     0
 DeviceProtection_has_error      0
 DeviceProtection_word_count     0
 DeviceProtection_len            0
 OnlineBackup_has_premium        0
 OnlineBackup_has_cancel         0
 dtype: int64,
 0)

In [21]:
spec = config.get("null_fill_with_flag", {})
print(spec.get("cols_numeric_zero"))

['TotalCharges', 'avg_charge_per_month']
